# 🪸 OpenClaw on Google Colab

Run your own **OpenClaw** AI gateway on a free Colab CPU instance.

---

### Before you start: Add your API key(s) to Colab Secrets

1. Click the **🔑 key icon** in the left sidebar (or go to **Runtime → Secrets**)
2. Click **"+ Add new secret"**
3. Add one or more of these (name must match exactly):

| Secret Name | Where to get it |
|---|---|
| `ANTHROPIC_API_KEY` | [console.anthropic.com/api-keys](https://console.anthropic.com/settings/keys) |
| `OPENAI_API_KEY` | [platform.openai.com/api-keys](https://platform.openai.com/api-keys) |
| `GEMINI_API_KEY` | [aistudio.google.com/apikey](https://aistudio.google.com/apikey) |
| `OPENROUTER_API_KEY` | [openrouter.ai/keys](https://openrouter.ai/keys) |

4. Toggle **"Notebook access"** ON for each secret

> ℹ️  You only need **one** provider. Your key stays in Colab's encrypted storage — it never appears in the notebook code.

---

Once your secret(s) are set, just **run each step below**. That's it.

---
## Step 1 — Install Node.js & OpenClaw

In [ ]:
#@title ▶️ Click to install (takes ~60 seconds)
%%bash
set -euo pipefail

# --- Node.js 22 ---
if command -v node &>/dev/null; then
  NODE_MAJOR=$(node -v | sed 's/v\([0-9]*\).*/\1/')
  if [ "$NODE_MAJOR" -ge 22 ]; then
    echo "✅ Node.js $(node -v) already installed"
  else
    echo "Upgrading Node.js..."
    curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash - &>/dev/null
    sudo apt-get install -y nodejs &>/dev/null
    echo "✅ Node.js $(node -v) installed"
  fi
else
  echo "Installing Node.js 22..."
  curl -fsSL https://deb.nodesource.com/setup_22.x | sudo -E bash - &>/dev/null
  sudo apt-get install -y nodejs &>/dev/null
  echo "✅ Node.js $(node -v) installed"
fi

# --- OpenClaw ---
if command -v openclaw &>/dev/null; then
  echo "✅ OpenClaw already installed"
else
  echo "Installing OpenClaw from npm..."
  npm install -g openclaw@latest 2>&1 | tail -3
  echo "✅ OpenClaw installed"
fi

echo
echo "Ready to go."

---
## Step 2 — Load API Keys & Choose Model

In [ ]:
#@title ▶️ Click to detect your keys and pick a model
import os
import secrets
import ipywidgets as widgets
from IPython.display import display, HTML

# --- Load secrets from Colab's secure storage ---
try:
    from google.colab import userdata
    _colab_secrets = True
except ImportError:
    _colab_secrets = False

SECRET_NAMES = [
    'ANTHROPIC_API_KEY',
    'OPENAI_API_KEY',
    'GEMINI_API_KEY',
    'OPENROUTER_API_KEY',
]

found_keys = {}
for name in SECRET_NAMES:
    val = None
    if _colab_secrets:
        try:
            val = userdata.get(name)
        except Exception:
            pass
    if not val:
        val = os.environ.get(name)
    if val and val.strip():
        found_keys[name] = val.strip()
        os.environ[name] = val.strip()

if not found_keys:
    display(HTML('''
    <div style="background:#fff3cd; border:1px solid #ffc107; border-radius:8px; padding:16px; margin:8px 0;">
      <b>⚠️ No API keys found!</b><br><br>
      Add at least one key in the <b>🔑 Secrets</b> panel (left sidebar):<br>
      <code>ANTHROPIC_API_KEY</code>, <code>OPENAI_API_KEY</code>,
      <code>GEMINI_API_KEY</code>, or <code>OPENROUTER_API_KEY</code><br><br>
      Make sure <b>"Notebook access"</b> is toggled ON, then re-run this cell.
    </div>
    '''))
else:
    # --- Build model options based on available keys ---
    MODELS = {}

    if 'ANTHROPIC_API_KEY' in found_keys:
        MODELS['claude-opus-4-6   (Anthropic — strongest reasoning)'] = 'claude-opus-4-6'
        MODELS['claude-sonnet-4-6 (Anthropic — fast & capable)'] = 'claude-sonnet-4-6'

    if 'GEMINI_API_KEY' in found_keys:
        MODELS['gemini-3-pro-preview   (Google — large context)'] = 'gemini-3-pro-preview'
        MODELS['gemini-3-flash-preview (Google — fast & free-tier friendly)'] = 'gemini-3-flash-preview'

    if 'OPENAI_API_KEY' in found_keys:
        MODELS['gpt-5.2 (OpenAI — latest)'] = 'gpt-5.2'

    if 'OPENROUTER_API_KEY' in found_keys:
        MODELS['OpenRouter (auto — OpenClaw picks the best available)'] = 'auto'

    # Show detected keys
    providers = [k.replace('_API_KEY', '').replace('_', ' ').title() for k in found_keys]
    display(HTML(f'''
    <div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:16px; margin:8px 0;">
      ✅ <b>Keys detected:</b> {', '.join(providers)}
    </div>
    '''))

    # Model selector dropdown
    model_options = list(MODELS.keys())
    model_dropdown = widgets.Dropdown(
        options=model_options,
        value=model_options[0],
        description='Model:',
        style={'description_width': '60px'},
        layout=widgets.Layout(width='500px'),
    )
    display(model_dropdown)

    # Store selection for later use
    def on_model_change(change):
        os.environ['OPENCLAW_MODEL'] = MODELS[change['new']]

    model_dropdown.observe(on_model_change, names='value')
    os.environ['OPENCLAW_MODEL'] = MODELS[model_options[0]]

    # --- Generate gateway token ---
    gateway_token = secrets.token_hex(32)
    os.environ['OPENCLAW_GATEWAY_TOKEN'] = gateway_token

    # --- Prepare state directory ---
    OPENCLAW_STATE = '/content/openclaw_state'
    os.makedirs(OPENCLAW_STATE, exist_ok=True)
    os.environ['OPENCLAW_STATE_DIR'] = OPENCLAW_STATE

    # Write .env for the gateway process
    env_path = os.path.join(OPENCLAW_STATE, '.env')
    with open(env_path, 'w') as f:
        f.write(f'OPENCLAW_GATEWAY_TOKEN={gateway_token}\n')
        for k, v in found_keys.items():
            f.write(f'{k}={v}\n')

    print(f'\nSelected model: {os.environ["OPENCLAW_MODEL"]}')
    print(f'Change the dropdown above before running Step 3 if you want a different model.')

---
## Step 3 — Start OpenClaw

In [ ]:
#@title ▶️ Click to launch the gateway
import subprocess
import os
import time

OPENCLAW_STATE = os.environ.get('OPENCLAW_STATE_DIR', '/content/openclaw_state')
MODEL = os.environ.get('OPENCLAW_MODEL', 'claude-opus-4-6')

# Build environment
env = os.environ.copy()
env['OPENCLAW_STATE_DIR'] = OPENCLAW_STATE
env['HOME'] = '/content'
env['NODE_ENV'] = 'production'
env['OPENCLAW_SKIP_CHANNELS'] = '1'

# Symlink state dir
home_openclaw = '/content/.openclaw'
if os.path.islink(home_openclaw):
    os.unlink(home_openclaw)
if not os.path.exists(home_openclaw):
    os.symlink(OPENCLAW_STATE, home_openclaw)

# Kill any previous gateway
try:
    subprocess.run(['pkill', '-f', 'openclaw.*gateway'], capture_output=True)
    time.sleep(2)
except Exception:
    pass

# Find openclaw
openclaw_bin = subprocess.run(['which', 'openclaw'], capture_output=True, text=True).stdout.strip()
if not openclaw_bin:
    print('\u274c OpenClaw not found. Run Step 1 first.')
elif not os.environ.get('OPENCLAW_GATEWAY_TOKEN'):
    print('\u274c No API keys configured. Run Step 2 first.')
else:
    log_file = open('/content/openclaw_gateway.log', 'w')
    proc = subprocess.Popen(
        [openclaw_bin, 'gateway', '--allow-unconfigured', '--bind', 'lan', '--port', '18789'],
        env=env,
        stdout=log_file,
        stderr=subprocess.STDOUT,
        preexec_fn=os.setsid
    )

    print(f'Starting OpenClaw with model: {MODEL}')
    print(f'PID: {proc.pid}')
    time.sleep(6)

    if proc.poll() is None:
        from IPython.display import HTML, display
        display(HTML(f'''
        <div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:16px; margin:12px 0;">
          <h3 style="margin-top:0">✅ OpenClaw is running!</h3>
          <table style="border:none; font-size:14px;">
            <tr><td><b>Model:</b></td><td><code>{MODEL}</code></td></tr>
            <tr><td><b>Dashboard:</b></td><td><code>http://localhost:18789</code></td></tr>
            <tr><td><b>Logs:</b></td><td><code>/content/openclaw_gateway.log</code></td></tr>
            <tr><td><b>Token:</b></td><td><code>{os.environ.get('OPENCLAW_GATEWAY_TOKEN','')[:16]}...</code></td></tr>
          </table>
        </div>
        '''))
    else:
        print('\n\u274c Gateway failed to start. Logs:')
        with open('/content/openclaw_gateway.log') as f:
            print(f.read()[-3000:])

---
## Optional: Expose via ngrok

Access OpenClaw from your phone or connect messaging channels.

Add `NGROK_AUTH_TOKEN` to your Colab Secrets (get one free at [ngrok.com](https://ngrok.com)), then run this cell.

In [ ]:
#@title ▶️ Click to open ngrok tunnel
import os

ngrok_token = None
try:
    from google.colab import userdata
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
except Exception:
    ngrok_token = os.environ.get('NGROK_AUTH_TOKEN')

if not ngrok_token:
    from IPython.display import HTML, display
    display(HTML('''
    <div style="background:#fff3cd; border:1px solid #ffc107; border-radius:8px; padding:16px;">
      <b>⚠️ No ngrok token found.</b><br>
      Add <code>NGROK_AUTH_TOKEN</code> to your <b>🔑 Secrets</b> panel and re-run this cell.
    </div>
    '''))
else:
    import subprocess
    subprocess.run(['pip', 'install', 'pyngrok', '-q'], check=True)

    from pyngrok import ngrok, conf
    conf.get_default().auth_token = ngrok_token.strip()

    tunnel = ngrok.connect(18789, 'http')
    from IPython.display import HTML, display
    display(HTML(f'''
    <div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:16px;">
      <h3 style="margin-top:0">✅ ngrok tunnel active</h3>
      <b>Public URL:</b> <a href="{tunnel.public_url}" target="_blank">{tunnel.public_url}</a><br>
      <b>Token:</b> <code>{os.environ.get('OPENCLAW_GATEWAY_TOKEN','')[:16]}...</code>
    </div>
    '''))

---
## Optional: Chat UI (Gradio)

In [ ]:
#@title ▶️ Click to launch chat interface
!pip install gradio requests -q

import gradio as gr
import requests
import os

GATEWAY_URL = 'http://localhost:18789'
TOKEN = os.environ.get('OPENCLAW_GATEWAY_TOKEN', '')


def send_message(message, history):
    try:
        r = requests.get(f'{GATEWAY_URL}/health', timeout=5)
        if r.status_code != 200:
            return 'Gateway is not responding. Run Step 3 first.'
    except Exception:
        return 'Gateway is not running. Run Step 3 first.'

    try:
        r = requests.post(
            f'{GATEWAY_URL}/api/chat',
            headers={'Content-Type': 'application/json', 'Authorization': f'Bearer {TOKEN}'},
            json={'message': message, 'channel': 'webchat'},
            timeout=120,
        )
        if r.status_code == 200:
            data = r.json()
            return data.get('reply', data.get('message', str(data)))
        return f'Error {r.status_code}: {r.text[:500]}'
    except requests.exceptions.Timeout:
        return 'Request timed out. The model may still be processing.'
    except Exception as e:
        return f'Error: {e}'


gr.ChatInterface(
    fn=send_message,
    type='messages',
    title='OpenClaw Chat',
    examples=['Hello! What can you do?', 'What model are you running?'],
).launch(share=False, debug=False)

---
## Optional: Google Drive Persistence

Data is stored locally by default (lost on runtime recycle). Run this to persist to Google Drive.

**Note:** This will request Google Drive access.

In [ ]:
#@title ▶️ Click to enable Google Drive storage
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')

DRIVE_STATE = '/content/drive/MyDrive/openclaw/state'
LOCAL_STATE = '/content/openclaw_state'

os.makedirs(DRIVE_STATE, exist_ok=True)

# Copy existing local state to Drive
if os.path.exists(LOCAL_STATE):
    for item in os.listdir(LOCAL_STATE):
        src = os.path.join(LOCAL_STATE, item)
        dst = os.path.join(DRIVE_STATE, item)
        if os.path.isfile(src):
            shutil.copy2(src, dst)
        elif os.path.isdir(src) and not os.path.exists(dst):
            shutil.copytree(src, dst)

os.environ['OPENCLAW_STATE_DIR'] = DRIVE_STATE

home_openclaw = '/content/.openclaw'
if os.path.islink(home_openclaw):
    os.unlink(home_openclaw)
os.symlink(DRIVE_STATE, home_openclaw)

from IPython.display import HTML, display
display(HTML(f'''
<div style="background:#d4edda; border:1px solid #28a745; border-radius:8px; padding:16px;">
  ✅ <b>Google Drive persistence enabled.</b><br>
  State: <code>{DRIVE_STATE}</code><br><br>
  Re-run Step 3 to restart the gateway with Drive storage.
</div>
'''))

---
## Utilities

In [ ]:
#@title 📜 View gateway logs
!tail -50 /content/openclaw_gateway.log 2>/dev/null || echo 'No log file yet. Run Step 3 first.'

In [ ]:
#@title 🟢 Check gateway status
!pgrep -fa 'openclaw.*gateway' || echo '❌ Gateway is NOT running. Run Step 3.'

In [ ]:
#@title ⏹️ Stop the gateway
!pkill -f 'openclaw.*gateway' && echo '✅ Gateway stopped.' || echo 'No gateway process found.'